# DL 전처리 파이프라인 테스트

이 노트북은 `src/DL/preprocessing.py`의 전처리 과정을 단계별로 실행하고 확인합니다.

## 전처리 단계
1. 데이터 로드
2. 시간축 정합 (1분 간격)
3. 결측치 처리 (단기: ffill, 중기: EWMA, 장기: EWMA)
4. 이상치 처리 (EWMA로 대체)
5. 리샘플링 (1시간)
6. 특성 생성

In [1]:
import sys
from pathlib import Path

# 프로젝트 루트 추가
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.DL.preprocessing import (
    impute_missing, 
    handle_outliers, 
    resample_data,
    ImputationConfig, 
    OutlierConfig
)
from src.DL.data_loader import load_raw_data, set_datetime_index
from src.DL.features import create_features

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print("✓ 모듈 임포트 완료")

✓ 모듈 임포트 완료


## 1. 데이터 로드

In [13]:
# 데이터 디렉토리
data_dir = project_root / "data" / "actual"

# 원본 데이터 로드
dfs = load_raw_data(data_dir)

print(f"로드된 파일: {list(dfs.keys())}")
for name, df in dfs.items():
    print(f"  {name}: {df.shape}")

로드된 파일: ['flow', 'tms', 'aws_368', 'aws_541', 'aws_569']
  flow: (130732, 6)
  tms: (514551, 7)
  aws_368: (781875, 10)
  aws_541: (830880, 10)
  aws_569: (780790, 10)


In [17]:
for name, df in dfs.items():
    if name == "flow" :
        dfs[name] = df.drop(columns=["data_save_dt"])
    if name.startswith("aws_"):
        cols_to_drop = ["YYMMDDHHMI_368","YYMMDDHHMI_541","YYMMDDHHMI_569","STN_368","STN_541","STN_569"]
        dfs[name] = df.drop(columns=cols_to_drop, errors="ignore")
    

In [18]:
for df in dfs.values():
    print(df.head())

   flow_TankA  flow_TankB  level_TankA  level_TankB          SYS_TIME
0    230.4000    230.2500      3.55225      3.54900  2025-09-02 23:53
1    229.9875    230.1000      3.54700      3.54575  2025-09-02 23:54
2    229.5000    230.0625      3.54825      3.54475  2025-09-02 23:55
3    229.8375    230.4750      3.54125      3.54250  2025-09-02 23:56
4    229.9125    230.0250      3.54050      3.54050  2025-09-02 23:57
           SYS_TIME  TOC_VU  PH_VU  SS_VU  FLUX_VU   TN_VU  TP_VU
0  2024-08-26 15:09     4.1    7.2    0.9   3912.0  11.923  0.039
1  2024-08-26 15:10     4.1    7.2    0.8   3917.0  11.923  0.039
2  2024-08-26 15:11     4.1    7.1    0.9   3923.0  11.923  0.039
3  2024-08-26 15:12     4.1    7.1    0.9   3927.0  11.923  0.039
4  2024-08-26 15:13     4.1    7.1    2.2   3932.0  11.923  0.039
   TA_368  RN_15m_368  RN_60m_368  RN_12H_368  RN_DAY_368  HM_368  TD_368  \
0    28.3         0.0         0.0         0.0         0.0    72.8    23.0   
1    28.3         0.0         

## 2. 시간축 정합 (1분 간격)

각 데이터를 DatetimeIndex로 설정하고 1분 간격으로 정합합니다.

In [19]:
# 시간 컬럼 매핑
time_col_map = {
    "flow": "SYS_TIME",
    "tms": "SYS_TIME",
    "aws_368": "datetime",
    "aws_541": "datetime",
    "aws_569": "datetime"
}

# DatetimeIndex 설정
aligned_dfs = {}
for name, df in dfs.items():
    if name in time_col_map:
        time_col = time_col_map[name]
        df_aligned = set_datetime_index(df, time_col)
        
        # 1분 간격으로 리샘플링 (시간 정합)
        df_aligned = df_aligned.resample("1min").mean()
        
        aligned_dfs[name] = df_aligned
        print(f"{name}: {df_aligned.shape}")

print("\n✓ 시간축 정합 완료")

flow: (131687, 4)
tms: (573975, 6)
aws_368: (781921, 7)
aws_541: (830880, 7)
aws_569: (780790, 7)

✓ 시간축 정합 완료


In [ ]:
# FLOW 데이터에 Q_in 생성
if "flow" in aligned_dfs:
    df_flow = aligned_dfs["flow"]
    if "flow_TankA" in df_flow.columns and "flow_TankB" in df_flow.columns:
        df_flow["Q_in"] = df_flow["flow_TankA"] + df_flow["flow_TankB"]
        print("✓ Q_in 생성 완료")

# 모델별로 데이터 병합
merged_dfs = {}

# flow 모델: flow + aws 3개
merged_dfs["flow"] = pd.concat([
    aligned_dfs["flow"], 
    aligned_dfs["aws_368"], 
    aligned_dfs["aws_541"], 
    aligned_dfs["aws_569"]
], axis=1, sort=False)

# modelA: tms + aws_368
merged_dfs["modelA"] = pd.concat([
    aligned_dfs["tms"], 
    aligned_dfs["aws_368"], 
    aligned_dfs["aws_541"], 
    aligned_dfs["aws_569"]
], axis=1, sort=False)

# modelB: tms + aws_541
merged_dfs["modelB"] = pd.concat([
    aligned_dfs["tms"], 
    aligned_dfs["aws_368"], 
    aligned_dfs["aws_541"], 
    aligned_dfs["aws_569"]
], axis=1, sort=False)

# modelC: tms + aws_569
merged_dfs["modelC"] = pd.concat([
    aligned_dfs["tms"], 
    aligned_dfs["aws_368"], 
    aligned_dfs["aws_541"], 
    aligned_dfs["aws_569"]
], axis=1, sort=False)

print("\n병합된 데이터 shape:")
for model_name, df in merged_dfs.items():
    print(f"  {model_name}: {df.shape}")
    print(f"    컬럼: {df.columns.tolist()}")

✓ Q_in 생성 완료


## 3. 결측치 처리

- 단기 (1-3시간): Forward Fill
- 중기 (4-12시간): EWMA (span=6)
- 장기 (12시간+): EWMA (span=24)

In [ ]:
# 결측치 처리 설정
config_impute = ImputationConfig(
    short_term_hours=3,
    medium_term_hours=12,
    long_term_hours=48,
    ewma_span=6
)

# 각 모델별로 결측치 처리
imputed_dfs = {}
mask_imputed_dfs = {}

for model_name, df in merged_dfs.items():
    print(f"\n{'='*60}")
    print(f"{model_name} 결측치 처리")
    print(f"{'='*60}")
    print(f"결측치 처리 전:")
    print(f"  총 결측치: {df.isna().sum().sum()}")
    print(f"  결측치 있는 컬럼: {(df.isna().sum() > 0).sum()}개")
    
    df_imputed, mask_imputed = impute_missing(
        df,
        freq="1min",
        config=config_impute
    )
    
    imputed_dfs[model_name] = df_imputed
    mask_imputed_dfs[model_name] = mask_imputed
    
    print(f"\n결측치 처리 후:")
    print(f"  총 결측치: {df_imputed.isna().sum().sum()}")
    print(f"  데이터 shape: {df_imputed.shape}")
    print(f"  마스크 shape: {mask_imputed.shape}")

print("\n✓ 모든 모델 결측치 처리 완료")

In [ ]:
# 결측치 처리 요약
imputation_summary = []
for col in df_merged.columns:
    if col in df_imputed.columns:
        summary = {
            'column': col,
            'original_missing': mask_imputed.get(f'{col}_is_missing', pd.Series(0)).sum(),
            'ffill': mask_imputed.get(f'{col}_imputed_ffill', pd.Series(0)).sum(),
            'ewma': mask_imputed.get(f'{col}_imputed_ewma', pd.Series(0)).sum(),
            'long_ewma': mask_imputed.get(f'{col}_imputed_long_ewma', pd.Series(0)).sum()
        }
        imputation_summary.append(summary)

df_imputation_summary = pd.DataFrame(imputation_summary)
df_imputation_summary = df_imputation_summary[df_imputation_summary['original_missing'] > 0]

print("결측치 처리 요약 (결측치가 있었던 컬럼만):")
print(df_imputation_summary.to_string(index=False))

결측치 처리 요약 (결측치가 있었던 컬럼만):
     column  original_missing  ffill  ewma  long_ewma
 flow_TankA            700148   1135     0     699013
 flow_TankB            700148   1135     0     699013
level_TankA            700148   1135     0     699013
level_TankB            700148   1135     0     699013
       Q_in            700148   1135     0     699013
     TA_368             59899  10387   607      48905
 RN_15m_368             61641  11128  1608      48905
 RN_60m_368             61902  11299  1698      48905
 RN_12H_368             66868  11510  3080      52278
 RN_DAY_368             61556  11067  1584      48905
     HM_368             59899  10387   607      48905
     TD_368             59899  10387   607      48905
     TA_541             61634   2337   831      58466
 RN_15m_541             67564   3581  2398      61585
 RN_60m_541             67953   3628  2533      61792
 RN_12H_541             74259   4617  1252      68390
 RN_DAY_541             67290   3425  2362      61503
  

## 4. 이상치 처리

- 탐지: 도메인 지식 + 통계적 방법 (IQR)
- 대체: EWMA (span=12)

In [27]:
# 이상치 처리
config_outlier = OutlierConfig(
    method="iqr",
    iqr_threshold=1.5,
    zscore_threshold=3.0,
    require_both=True
)

df_outlier_handled, mask_outlier = handle_outliers(
    df_imputed,
    config=config_outlier,
    ewma_span=12
)

print(f"이상치 처리 후:")
print(f"  데이터 shape: {df_outlier_handled.shape}")
print(f"  마스크 shape: {mask_outlier.shape}")

print("\n✓ 이상치 처리 완료")

이상치 처리 후:
  데이터 shape: (830880, 27)
  마스크 shape: (830880, 83)

✓ 이상치 처리 완료


In [ ]:
# 이상치 처리 요약
outlier_summary = []
for col in df_imputed.columns:
    if f'{col}_outlier_final' in mask_outlier.columns:
        summary = {
            'column': col,
            'domain': mask_outlier.get(f'{col}_outlier_domain', pd.Series(0)).sum(),
            'statistical': mask_outlier.get(f'{col}_outlier_statistical', pd.Series(0)).sum(),
            'final': mask_outlier[f'{col}_outlier_final'].sum(),
            'replaced_ewma': mask_outlier.get(f'{col}_outlier_replaced_ewma', pd.Series(0)).sum()
        }
        outlier_summary.append(summary)

df_outlier_summary = pd.DataFrame(outlier_summary)
df_outlier_summary = df_outlier_summary[df_outlier_summary['final'] > 0]

print("이상치 처리 요약 (이상치가 있었던 컬럼만):")
print(df_outlier_summary.to_string(index=False))

## 5. 리샘플링 (1시간)

1분 간격 데이터를 1시간 간격으로 리샘플링합니다.

In [ ]:
# 리샘플링
df_resampled = resample_data(
    df_outlier_handled,
    freq="1h",
    agg="mean"
)

print(f"리샘플링 후:")
print(f"  shape: {df_resampled.shape}")
print(f"  시간 범위: {df_resampled.index.min()} ~ {df_resampled.index.max()}")
print(f"  총 시간: {(df_resampled.index.max() - df_resampled.index.min()).days}일")

print("\n✓ 리샘플링 완료")

## 6. 특성 생성 (Flow 모델)

Flow 모델용 특성을 생성합니다.

In [ ]:
# 사용 가능한 컬럼 확인
print("사용 가능한 컬럼:")
print(f"  총 {len(df_resampled.columns)}개")
print(f"  컬럼 목록: {df_resampled.columns.tolist()}")

# TMS 컬럼 확인
tms_cols = ["TOC_VU", "PH_VU", "SS_VU", "FLUX_VU", "TN_VU", "TP_VU"]
has_tms = [col for col in tms_cols if col in df_resampled.columns]
print(f"\nTMS 컬럼: {has_tms if has_tms else '없음'}")

# FLOW 컬럼 확인
flow_cols = ["level_TankA", "level_TankB", "Q_in"]
has_flow = [col for col in flow_cols if col in df_resampled.columns]
print(f"FLOW 컬럼: {has_flow if has_flow else '없음'}")

# 강수 컬럼 확인
rain_cols = [f"RN_15m_{sid}" for sid in ["368", "541", "569"]]
has_rain = [col for col in rain_cols if col in df_resampled.columns]
print(f"강수 컬럼: {has_rain if has_rain else '없음'}")

In [ ]:
# 특성 생성 (Flow 모델)
model_mode = "flow"
target_cols = ["Q_in"]

print(f"\n특성 생성 시작: {model_mode} 모드")
print(f"타겟 컬럼: {target_cols}")

df_features = create_features(
    df_resampled,
    model_mode=model_mode,
    target_cols=target_cols,
    add_time=True,
    add_sin_cos=True,
    lag_hours=[1, 2, 3, 6, 12, 24],
    rolling_windows=[3, 6, 12, 24],
    rolling_stats=["mean", "std"]
)

print(f"\n특성 생성 후:")
print(f"  shape: {df_features.shape}")
print(f"  생성된 특성 수: {len(df_features.columns) - len(df_resampled.columns)}개")

print("\n✓ 특성 생성 완료")

In [ ]:
# NaN 확인
nan_counts = df_features.isna().sum()
cols_with_nan = nan_counts[nan_counts > 0]

print(f"\nNaN 확인:")
print(f"  NaN이 있는 컬럼: {len(cols_with_nan)}개")
print(f"  총 NaN 개수: {nan_counts.sum()}")

if len(cols_with_nan) > 0:
    print(f"\n  상위 10개 컬럼:")
    for col, count in cols_with_nan.head(10).items():
        print(f"    {col}: {count}")

## 7. 시각화

전처리 전후 데이터를 시각화합니다.

In [ ]:
# Q_in 시각화 (있는 경우)
if "Q_in" in df_features.columns:
    fig, axes = plt.subplots(3, 1, figsize=(15, 10))
    
    # 원본 (리샘플링 후)
    if "Q_in" in df_resampled.columns:
        axes[0].plot(df_resampled.index, df_resampled["Q_in"], alpha=0.7)
        axes[0].set_title("Q_in - 리샘플링 후 (1시간)")
        axes[0].set_ylabel("Q_in")
        axes[0].grid(True, alpha=0.3)
    
    # 특성 생성 후
    axes[1].plot(df_features.index, df_features["Q_in"], alpha=0.7, color='orange')
    axes[1].set_title("Q_in - 특성 생성 후")
    axes[1].set_ylabel("Q_in")
    axes[1].grid(True, alpha=0.3)
    
    # 분포 비교
    if "Q_in" in df_resampled.columns:
        axes[2].hist(df_resampled["Q_in"].dropna(), bins=50, alpha=0.5, label='리샘플링 후')
    axes[2].hist(df_features["Q_in"].dropna(), bins=50, alpha=0.5, label='특성 생성 후', color='orange')
    axes[2].set_title("Q_in 분포 비교")
    axes[2].set_xlabel("Q_in")
    axes[2].set_ylabel("빈도")
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Q_in 컬럼이 없습니다.")

## 8. 최종 데이터 저장 (선택사항)

In [ ]:
# 전처리된 데이터 저장
save_path = project_root / "data" / "processed" / "flow_preprocessed.csv"

# 저장 여부 확인
save_data = False  # True로 변경하면 저장됨

if save_data:
    df_features.to_csv(save_path)
    print(f"✓ 데이터 저장 완료: {save_path}")
else:
    print("데이터 저장 안 함 (save_data=False)")

## 요약

전처리 파이프라인이 성공적으로 완료되었습니다.

In [ ]:
print("="*60)
print("전처리 파이프라인 요약")
print("="*60)
print(f"1. 데이터 로드: {len(dfs)}개 파일")
print(f"2. 시간축 정합: {df_merged.shape}")
print(f"3. 결측치 처리: {df_imputed.shape}")
print(f"4. 이상치 처리: {df_outlier_handled.shape}")
print(f"5. 리샘플링: {df_resampled.shape}")
print(f"6. 특성 생성: {df_features.shape}")
print(f"\n최종 특성 수: {len(df_features.columns)}개")
print(f"최종 샘플 수: {len(df_features)}개")
print(f"NaN이 있는 컬럼: {len(cols_with_nan)}개")
print("="*60)